In [24]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [25]:
#Create MILP model object
mymodel = glp.Solver('GolfballMania', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [26]:
inf = mymodel.infinity()

         # name: supply capacity (thousands), fixed cost (thousand $), unit shipping cost ($) to each RDC
source = {'Cincinnati': (40, 400, [2, 12, 7, 8]),    # Cincinnati
          'Pittsburgh': (20, 200, [15, 13, 14, 10]),    # Pittsburgh
          'Chicago': (18, 180, [15, 13, 14, 17]),    # Chicago
          'Louisville': (25, 250, [8, 17, 9, 12]),   # Louisville
          'Indianapolis': (30, 300, [8, 13, 10, 11])} #Indianapolis

        # name: demand quantity required (thousands)
demand = {'Los Angeles': 36, 'Toledo': 22, 'Dayton': 28, 'Columbus': 20}  

mymodel.Objective().SetMinimization    # minimize total plant and transportation cost

<bound method Objective.SetMinimization of <ortools.linear_solver.pywraplp.Objective; proxy of <Swig Object of type 'operations_research::MPObjective *' at 0x000001EC3534E660> >>

In [27]:
# create binary Plant selection variables and a dictionary to access them
select = dict()
for s in source:
    select[s] = mymodel.IntVar(0,1,s)
    (b, f, coeff_lst) = source[s]
    mymodel.Objective().SetCoefficient(select[s], f)

In [28]:
# create shipment variables from Plants to RDCs and a dictionary to access them
ship = dict()
for s in source:
    ship[s] = dict()
    (b, f, coeff_lst) = source[s]
    for (d,c) in zip(demand,coeff_lst):
        ship[s][d] = mymodel.NumVar(0, inf, s + '.' + d)
        mymodel.Objective().SetCoefficient(ship[s][d],c)

In [29]:
# create source/supply constraints
for s in source:
    (b,f,coeff_lst) = source[s]
    constr = mymodel.Constraint(-inf, 0, s)
    constr.SetCoefficient(select[s], -b)
    for d in demand:
        constr.SetCoefficient(ship[s][d], 1)

In [30]:
# create demand constraints
for d in demand:
    lb = ub = demand[d]
    constr = mymodel.Constraint(lb,ub,d)
    for s in source:
        constr.SetCoefficient(ship[s][d], 1)

In [31]:
lpt.print_model(mymodel)

Variables:
Cincinnati, Pittsburgh, Chicago, Louisville, Indianapolis, Cincinnati.Los Angeles, Cincinnati.Toledo, Cincinnati.Dayton, Cincinnati.Columbus, Pittsburgh.Los Angeles, Pittsburgh.Toledo, Pittsburgh.Dayton, Pittsburgh.Columbus, Chicago.Los Angeles, Chicago.Toledo, Chicago.Dayton, Chicago.Columbus, Louisville.Los Angeles, Louisville.Toledo, Louisville.Dayton, Louisville.Columbus, Indianapolis.Los Angeles, Indianapolis.Toledo, Indianapolis.Dayton, Indianapolis.Columbus 

minimize: 400.0*Cincinnati + 200.0*Pittsburgh + 180.0*Chicago + 250.0*Louisville + 300.0*Indianapolis + 2.0*Cincinnati.Los Angeles + 12.0*Cincinnati.Toledo + 7.0*Cincinnati.Dayton + 8.0*Cincinnati.Columbus + 15.0*Pittsburgh.Los Angeles + 13.0*Pittsburgh.Toledo + 14.0*Pittsburgh.Dayton + 10.0*Pittsburgh.Columbus + 15.0*Chicago.Los Angeles + 13.0*Chicago.Toledo + 14.0*Chicago.Dayton + 17.0*Chicago.Columbus + 8.0*Louisville.Los Angeles + 17.0*Louisville.Toledo + 9.0*Louisville.Dayton + 12.0*Louisville.Columbus + 8.0

In [32]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f thousand' % mymodel.Objective().Value())
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%7s = %5.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 1906.00 thousand
Cincinnati =  1.00
Pittsburgh =  1.00
Chicago =  1.00
Indianapolis =  1.00
Cincinnati.Los Angeles = 36.00
Cincinnati.Dayton =  4.00
Pittsburgh.Columbus = 20.00
Chicago.Toledo = 16.00
Indianapolis.Toledo =  6.00
Indianapolis.Dayton = 24.00


In [33]:
# display all variable information
print('Variable    LB   Value    UB')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f' % (v.name(),v.lb(),v.solution_value(),v.ub()))

Variable    LB   Value    UB
Cincinnati    0.0    1.0    1.0
Pittsburgh    0.0    1.0    1.0
 Chicago    0.0    1.0    1.0
Louisville    0.0    0.0    1.0
Indianapolis    0.0    1.0    1.0
Cincinnati.Los Angeles    0.0   36.0    inf
Cincinnati.Toledo    0.0    0.0    inf
Cincinnati.Dayton    0.0    4.0    inf
Cincinnati.Columbus    0.0    0.0    inf
Pittsburgh.Los Angeles    0.0    0.0    inf
Pittsburgh.Toledo    0.0    0.0    inf
Pittsburgh.Dayton    0.0    0.0    inf
Pittsburgh.Columbus    0.0   20.0    inf
Chicago.Los Angeles    0.0    0.0    inf
Chicago.Toledo    0.0   16.0    inf
Chicago.Dayton    0.0    0.0    inf
Chicago.Columbus    0.0    0.0    inf
Louisville.Los Angeles    0.0    0.0    inf
Louisville.Toledo    0.0    0.0    inf
Louisville.Dayton    0.0    0.0    inf
Louisville.Columbus    0.0    0.0    inf
Indianapolis.Los Angeles    0.0    0.0    inf
Indianapolis.Toledo    0.0    6.0    inf
Indianapolis.Dayton    0.0   24.0    inf
Indianapolis.Columbus    0.0    0.0    inf


In [34]:
#display constraint information
print('Constraint    LB    Value  UB')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f' % (c.name(),c.lb(),lhs,c.ub()))

Constraint    LB    Value  UB
Cincinnati   -inf    0.0    0.0
Pittsburgh   -inf    0.0    0.0
   Chicago   -inf   -2.0    0.0
Louisville   -inf    0.0    0.0
Indianapolis   -inf    0.0    0.0
Los Angeles   36.0   36.0   36.0
    Toledo   22.0   22.0   22.0
    Dayton   28.0   28.0   28.0
  Columbus   20.0   20.0   20.0
